In [33]:
import pandas as pd
import os
import glob

In [ ]:
folder_path = ".raw_prices"

In [35]:
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

df_list = []

In [36]:
for file in all_files:
    # read each csv
    df = pd.read_csv(file)

    # get filename without extension
    file_name = os.path.splitext(os.path.basename(file))[0]

    # add filename column
    df["ticker"] = file_name

    df_list.append(df)

In [37]:
# merge everything into one big DataFrame
merged_df = pd.concat(df_list, ignore_index=True)

In [38]:
# remove dollar signs from close column
merged_df["Close/Last"] = df["Close/Last"].astype(str).str[1:]

# rename columns
merged_df = merged_df.rename(columns={"Close/Last": "close", "Date": "date"})
# remove columns
merged_df = merged_df.drop(columns=["Volume", "Open", "High", "Low"])

# reorder columns
merged_df = merged_df[["ticker", "date", "close"]]

merged_df = merged_df.astype({
    "ticker": "string",
    "date": "string",
    "close": float
})

# adding return
# compute daily returns
merged_df["ret"] = merged_df.groupby("ticker")["close"].pct_change()

/tmp/ipykernel_305344/3384496434.py:20: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  merged_df["ret"] = merged_df.groupby("ticker")["close"].pct_change()


In [39]:
merged_df.head()

,ticker,date,close,ret
0,AAPL,09/08/2025,109.85,NaN
1,AAPL,09/05/2025,109.23,-0.005644
2,AAPL,09/04/2025,112.40,0.029021
3,AAPL,09/03/2025,111.91,-0.004359
4,AAPL,09/02/2025,114.69,0.024841


In [40]:
# save DataFrame to csv
merged_df.to_csv("prices.csv", index=False)